In [ ]:
# Required Packages

%tensorflow_version 2.1.0
!pip install gradio

In [2]:
# Required Libraries

import numpy as np
import cv2
import imutils
import tensorflow as tf
from tensorflow import keras
import gradio as gr
from tensorflow.keras.models import load_model

In [19]:
# Loading Model 

model = load_model('/content/drive/MyDrive/Colab Notebooks/saved_models/multires_original.h5', compile=False)


In [20]:
def segmentation(inp):
                    
    inp = cv2.resize(inp, (256, 256))               # Resize
    inp = (inp.astype('float32')) / 255.            # Normalize
    test_input = inp

    # Predictions
    prediction_on_test = np.expand_dims(test_input, 0)            
    prediction_on_test = model.predict(prediction_on_test)     # Prediction
    prediction_on_test = prediction_on_test > 0.5               # Thresholding
    predicted_img = prediction_on_test[0,:,:,0]   

     # Extracting Contours
    predicted = predicted_img.astype(np.uint8)
    cnts = cv2.findContours(image=predicted, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
    contours = imutils.grab_contours(cnts)
    contoured = test_input.copy()
    contoured = (contoured * 255).astype(np.uint8)
    cv2.drawContours(image=contoured, contours=contours, contourIdx=-1, color=(255, 0, 0), thickness=1, lineType=cv2.LINE_AA)

     # Circumference of detected Mask
    if contours :
        a = "Polynya Detected"
        for i in range(len(contours)):
            circum = cv2.arcLength(contours[i], True)
            circum = round(circum,2)
            b = str(circum) + '\t' + "px"
    else:
        a = "No Polynya Detected"
        b = "0.0 px"

    return(contoured, a, b)


In [21]:
image = gr.Image(label = 'Input Image')
out1 = gr.Image(label = 'Result')
out2 = gr.Textbox(label = 'Label')
out3 = gr.Textbox(label = 'Circumference in Pixel Unit')

interface = gr.Interface(fn = segmentation, inputs = image, outputs = [out1, out2, out3], 
                         title= 'Polynya Detection', 
                         description= 'Let the system detect if there is a Polynya in your image.')


In [22]:
interface.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://19854.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x7fc0d2da3d50>,
 'http://127.0.0.1:7864/',
 'https://19854.gradio.app')